# Import code

In [ ]:
%matplotlib notebook

import numpy               as np
import tensorflow          as tf
import tensorflow_addons   as tfa
import matplotlib.pyplot   as plt
import pandas              as pd
import gudhi               as gd

from gudhi.differentiation import *

from gudhi.wasserstein     import wasserstein_distance
from mpl_toolkits.mplot3d  import Axes3D
from sklearn.metrics       import pairwise_distances

In [ ]:
my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
tf.config.experimental.set_visible_devices(devices=my_devices, device_type='CPU')
tf.config.experimental.set_visible_devices([], 'GPU')

# Point cloud application

In this first experiment, we optimize the point coordinates of an initial random point cloud so that 1-dimensional homology (holes) is maximized.

In [ ]:
Xinit = np.array(np.random.uniform(high=1., low=-1., size=(300,2)), dtype=np.float32)

In [ ]:
plt.figure()
plt.scatter(Xinit[:,0], Xinit[:,1])
plt.title("Point cloud at epoch 0")
plt.show()

In [ ]:
X = tf.Variable(initial_value=Xinit, trainable=True)
model = RipsModel(X=X, mel=12., dim=1, card=50)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)

In [ ]:
losses, dgms = [], []
for epoch in range(1000+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
        
        # Loss is the opposite of the sum of squares of distances to the diagonal
        # plus a penalty for points outside of [0,1]^2
        loss = -tf.math.reduce_sum(tf.square(.5*(dgm[:,1]-dgm[:,0]))) + tf.reduce_sum(tf.maximum(tf.abs(X)-1, 0))
        
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    losses.append(loss.numpy())
    dgms.append(dgm)        

In [ ]:
plt.figure()
plt.scatter(model.X.numpy()[:,0], model.X.numpy()[:,1])
plt.title("Point cloud at epoch " + str(epoch))
plt.show()

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure()
for dg in dgms[:5:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker="D", alpha=0.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker="D", c="green")
plt.plot([-0.,.2], [-0.,.2])
plt.title("Optimized persistence diagrams")
plt.show()

# 2D image application

In the second experiment, we optimize the pixel values of an image so that the artificial noise is removed by minimizing 0-dimensional homology (connected components).

In [ ]:
I = np.array(pd.read_csv("datasets/diff/mnist_test.csv", header=None, sep=","), dtype=np.float32)
idx = np.argwhere(I[:,0] == 8)
image = np.reshape(-I[idx[8],1:], [28,28])
image = (image-image.min())/(image.max()-image.min())
image_clean = np.array(image)
image[2:5,2:5]        -= 0.6
image[25:27,25:27]    -= 0.6
image[25:27,2:5]      -= 0.6
image[1:4,24:26]      -= 0.6

In [ ]:
plt.figure()
plt.imshow(image, cmap='Greys')
plt.title("Image at epoch 0")
plt.show()

In [ ]:
X = tf.Variable(initial_value=np.array(image, dtype=np.float32), trainable=True)
model = CubicalModel(X, dim=0, card=100)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)

In [ ]:
losses, dgms, empty = [], [], np.empty([0,2])
alpha, gamma = 10., 0.
for epoch in range(3000+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
        
        # Loss is total persistence plus distance to clean image 
        loss = alpha * tf.math.reduce_sum(tf.abs(dgm[:,1]-dgm[:,0])) + \
               gamma * tf.math.reduce_sum(tf.math.minimum(tf.abs(model.X), tf.abs(1.-model.X)))
                       #tf.math.reduce_sum(tf.square(model.X-image_clean))
        
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    losses.append(loss.numpy())
    dgms.append(dgm) 

In [ ]:
plt.figure()
plt.imshow(model.X.numpy(), cmap='Greys')
plt.title("Image at epoch " + str(epoch))
plt.show()

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker="D", c="blue")
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker="D", alpha=0.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker="D", c="red")
plt.plot([0,1], [0,1])
plt.title("Optimized persistence diagrams")
plt.show()

# 3D shape application

In the third experiment, we optimize the vertex filtration values of a 3D shape so that the corresponding persistence diagram matches another constant one (in the Wasserstein distance).

In [ ]:
faces, coord = np.loadtxt("datasets/diff/human_faces", dtype=float)[:,1:], np.loadtxt("datasets/diff/human_coords", dtype=float)
stbase = gd.SimplexTree()
for i in range(len(faces)):
    stbase.insert(faces[i,:], -1e10)
f = open("datasets/diff/human_simplextree.txt", "w")
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write("\n")
f.close()

In [ ]:
Finit = coord[:,2]

In [ ]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap("rainbow")
ax = fig.add_subplot(111, projection="3d")
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=Finit[::step], s=2, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title("Function at epoch 0")
plt.show()

In [ ]:
F = tf.Variable(initial_value=np.array(Finit, dtype=np.float32), trainable=True)
model = SimplexTreeModel(F, stbase="datasets/diff/human_simplextree.txt", dim=0, card=50)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)

In [ ]:
losses, dgms = [], []
alpha, gamma = 1., .001
for epoch in range(1500+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
    
        # Loss is square Wasserstein distance between the current persistence diagram and the target
        loss = alpha * tf.square(wasserstein_distance(dgm, tf.constant(np.array([[-.98,-.03]], dtype=np.float32)),
                                 order=2, enable_autodiff=True))               
    
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    losses.append(loss.numpy())
    dgms.append(dgm)

In [ ]:
step = 1
fig = plt.figure()
cm = plt.cm.get_cmap("rainbow")
ax = fig.add_subplot(111, projection="3d")
sc = ax.scatter(coord[::step,0], coord[::step,1], coord[::step,2], c=F.numpy()[::step], s=1, 
                vmin=0, vmax=.75, cmap=cm)
x_limits, y_limits, z_limits = ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()
x_range, x_middle = abs(x_limits[1] - x_limits[0]), np.mean(x_limits)
y_range, y_middle = abs(y_limits[1] - y_limits[0]), np.mean(y_limits)
z_range, z_middle = abs(z_limits[1] - z_limits[0]), np.mean(z_limits)
plot_radius = 0.5*max([x_range, y_range, z_range])
ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
plt.title("Function at epoch " + str(epoch))
plt.show()

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker="D", c="blue")
for dg in dgms[:-1:2]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker="D", alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker="D", c="red")
plt.plot([-1,1], [-1,1])
plt.title("Optimized persistence diagrams")
plt.show()

# Regression application

In the fourth experiment, we optimize the coefficients of a linear regression model so that the persistence diagram of its sublevel sets is empty, except for the three most prominent points.

In [ ]:
n, p = 50, 100
betastar = np.concatenate([np.linspace(-1.,1.,33) for _ in range(3)] + [[-1.]])
X = np.random.multivariate_normal(mean=np.zeros(shape=[p]), cov=np.eye(p), size=n)
Y = np.matmul(X, betastar) + .05 * np.random.randn(n)
X, Y = np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)
stbase = gd.SimplexTree()
for i in range(p-1):
    stbase.insert([i,i+1], -1e10)
f = open("datasets/diff/beta_simplextree.txt", "w")
for (s,_) in stbase.get_filtration():
    for v in s:
        f.write(str(v) + " ")
    f.write("\n")
f.close()
betainit = np.random.uniform(low=-1., high=1., size=[p])
betainit[np.array([25,60,99])] = np.array([-1,-1,-1])

In [ ]:
plt.figure()
plt.plot(betastar)
plt.title("Ground-truth coefficients")
plt.show()

In [ ]:
plt.figure()
plt.plot(betainit)
plt.title("Coefficients at epoch 0")
plt.show()

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
model = SimplexTreeModel(beta, stbase="datasets/diff/beta_simplextree.txt", dim=0, card=100)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5)

In [ ]:
losses, dgms, betas = [], [], []
alpha, gamma, delta = 1, 1e4, 1e3
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
        
        # Loss is MSE plus the total persistence except for the three most prominent points
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
            
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])

beta_stdtop = -betas[-1]

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker="D", c="blue")
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker="D", alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker="D", c="red")
plt.plot([-1,1], [-1,1])
plt.title("Optimized persistence diagrams")
plt.show()

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
model = SimplexTreeModel(beta, stbase="datasets/diff/beta_simplextree.txt", dim=0, card=100)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5)

In [ ]:
losses, dgms, betas = [], [], []
alpha, gamma, delta = 1, 0, 1e3
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
        
        # Loss is MSE plus the total persistence except for the three most prominent points
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
            
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])
                 
beta_stdtot = -betas[-1]

In [ ]:
beta = tf.Variable(initial_value=np.array(betainit[:,np.newaxis], dtype=np.float32), trainable=True)
model = SimplexTreeModel(beta, stbase="datasets/diff/beta_simplextree.txt", dim=0, card=100)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5)

In [ ]:
losses, dgms, betas = [], [], []
alpha, gamma, delta = 2, 0, 0
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
        
        # Loss is MSE plus the total persistence except for the three most prominent points
        loss = alpha * tf.reduce_sum(tf.square(tf.matmul(X, -beta) - Y)) \
             + gamma * tf.reduce_sum(tf.abs(dgm[2:,1]-dgm[2:,0])) \
             + delta * tf.reduce_sum(tf.abs(beta[1:]-beta[:-1]))
             
            
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    losses.append(loss.numpy())
    dgms.append(dgm)
    betas.append(beta.numpy()[:,0])
    
beta_std = -betas[-1]

In [ ]:
plt.figure()
plt.plot(betastar, label="ground-truth")
plt.plot(beta_std, label="MSE")
plt.plot(beta_stdtot, label="MSE+TV")
plt.plot(beta_stdtop, label="MSE+TV+TOP")
plt.legend(loc="upper left")
plt.show()
print(np.linalg.norm(betastar-beta_std), 
      np.linalg.norm(betastar-beta_stdtot), 
      np.linalg.norm(betastar-beta_stdtop))

In [ ]:
print(np.square(np.matmul(X, beta_std) - Y).sum(), 
      np.square(np.matmul(X, beta_stdtot) - Y).sum(),
      np.square(np.matmul(X, beta_stdtop) - Y).sum())

MSEstd, MSEtop, MSEtot = [], [], []
for _ in range(1000):
    Xnew = np.random.multivariate_normal(mean=np.zeros(shape=[p]), cov=np.eye(p), size=n)
    Ynew = np.matmul(Xnew, betastar) #+ np.random.normal(0.,.05,n)
    mse_std = np.square(np.matmul(Xnew, beta_std) - Ynew).sum()
    mse_tot = np.square(np.matmul(Xnew, beta_stdtot) - Ynew).sum()
    mse_top = np.square(np.matmul(Xnew, beta_stdtop) - Ynew).sum()
    MSEstd.append(mse_std)
    MSEtot.append(mse_tot)
    MSEtop.append(mse_top)
    
plt.figure()
plt.boxplot([MSEstd, MSEtot, MSEtop], labels=["MSE", "MSE+TV", "MSE+TV+TOP"])
plt.title("MSE on random test sets")
plt.show()

# Noisy point cloud

In [ ]:
n, epsilon, nout = 100, .2, 3
x, y = np.cos(np.linspace(0,2*np.pi,n)), np.sin(np.linspace(0,2*np.pi,n))
np.random.seed(0)
ex, ey = np.random.uniform(low=-epsilon,high=epsilon,size=n), np.random.uniform(low=-epsilon,high=epsilon,size=n)
np.random.seed(0)
outliers = np.random.uniform(low=-.7, high=.7, size=(nout,2))

In [ ]:
plt.figure()
plt.scatter(x+ex, y+ey)
plt.scatter(outliers[:,0], outliers[:,1])
plt.title("Point cloud at epoch 0")
plt.show()

In [ ]:
st = gd.RipsComplex(distance_matrix=pairwise_distances(np.hstack([x[:,np.newaxis],y[:,np.newaxis]])), 
                    max_edge_length=2.).create_simplex_tree(max_dimension=2)
st.persistence()
D = np.array(st.persistence_intervals_in_dimension(0), dtype=np.float32)[:-1]

In [ ]:
plt.figure()
plt.scatter(D[:,0], D[:,1])
plt.plot([0,2], [0,2])
plt.show()

In [ ]:
Xinit = np.array(np.vstack([np.hstack([(x+ex)[:,np.newaxis], (y+ey)[:,np.newaxis]]),outliers]), dtype=np.float32)

In [ ]:
X = tf.Variable(initial_value=Xinit, trainable=True)
model = RipsModel(X=X, mel=2., dim=0, card=150)
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-1)

In [ ]:
losses, dgms = [], []
for epoch in range(100+1):
    
    with tf.GradientTape() as tape:
        
        # Compute persistence diagram
        dgm = model.call()
        
        # Loss is Wasserstein distance
        loss = tf.square(wasserstein_distance(dgm, tf.constant(D), order=2, enable_autodiff=True))
        
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    losses.append(loss.numpy())
    dgms.append(dgm)        

In [ ]:
plt.figure()
plt.scatter(model.X.numpy()[:,0], model.X.numpy()[:,1])
plt.title("Point cloud at epoch " + str(epoch))
plt.show()

In [ ]:
plt.figure()
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.figure()
plt.scatter(dgms[0][:,0], dgms[0][:,1], s=40, marker="D", c="blue")
for dg in dgms[:-1]:
    plt.scatter(dg[:,0], dg[:,1], s=20, marker="D", alpha=.1)
plt.scatter(dgms[-1][:,0], dgms[-1][:,1], s=40, marker="D", c="red")
plt.plot([-1,1], [-1,1])
plt.title("Optimized persistence diagrams")
plt.show()